![](https://raw.githubusercontent.com/Databricks-BR/workshop_agents/refs/heads/main/demo-main/img/header_workshop.png)


 Item | Description |
 --- | --- |
 **Objective** | Create different UC Functions |
 **Databricks Run Time** | DBR 16.4 LTS |
 **Language** | Python, Pyspark and SQL |


![](https://raw.githubusercontent.com/Databricks-BR/workshop_agents/refs/heads/main/demo-main/img/img/02_diagram.png)

## Create Functions

![](https://raw.githubusercontent.com/Databricks-BR/workshop_agents/refs/heads/main/demo-main/img/img/02_functions.png)

**SQL functions** are designed to simplify and streamline data processing within queries by allowing users to encapsulate reusable logic for calculations, value transformations, or aggregating results directly in SQL statements; within Databricks, these functions are governed by Unity Catalog, ensuring centralized, secure management and consistent governance across all assets.

### Get Store by ID

In [0]:
%sql
CREATE OR REPLACE FUNCTION vinicius_fialho_testes.workshop_ml_agentes.get_store_by_id(
  p_store_id STRING COMMENT 'The unique ID of the store to be searched.'
)
RETURNS TABLE(
  store STRING COMMENT 'Name or identifier of the store.',
  store_type STRING COMMENT 'Type or category of the store.',
  store_zip STRING COMMENT 'Zip code of the store.',
  retailer STRING COMMENT 'Name of the retailer the store belongs to.',
  store_lat_long STRING COMMENT 'Latitude and longitude coordinates of the store.'
)
COMMENT 'Function to retrieve store data based on the provided ID. It returns information from the mvw_inventory metric view.'
RETURN
  SELECT
    store,
    store_type,
    store_zip,
    retailer,
    store_lat_long
  FROM
    vinicius_fialho_testes.workshop_ml_agentes.dim_store
  WHERE store_id = p_store_id
  LIMIT 1;

In [0]:
%sql
SELECT * FROM vinicius_fialho_testes.workshop_ml_agentes.get_store_by_id("4823349147981451781")

### Calculate Lat and Long

In [0]:
%sql
CREATE OR REPLACE FUNCTION vinicius_fialho_testes.workshop_ml_agentes.geocode_address(address STRING)
RETURNS STRUCT<latitude DOUBLE, longitude DOUBLE>
LANGUAGE PYTHON
AS
$$
import http.client
import json
import urllib.parse

def geocode_via_nominatim(address):
    if not address:
        return (None, None)
        
    encoded_address = urllib.parse.quote(address)
    
    conn = http.client.HTTPSConnection('nominatim.openstreetmap.org')
    
    headers = { 
        'User-Agent': f'Databricks-Geocoding-Function/1.0',
        'Accept': 'application/json'
    } 
    
    # Corrected f-string syntax for request_path
    request_path = f'/search?q={encoded_address}&format=json&limit=1'
    
    try:
        conn.request('GET', request_path, headers=headers)
        
        response = conn.getresponse()
        
        data = json.loads(response.read().decode())
        
        if data and len(data) > 0:
            return (float(data[0]['lat']), float(data[0]['lon']))
        else:
            return (None, None)
    except Exception:
        return (None, None)
    finally:
        conn.close()

result = geocode_via_nominatim(address)
return {'latitude': result[0], 'longitude': result[1]}
$$;

In [0]:
%sql
WITH geocoded_location AS (
  SELECT 
    vinicius_fialho_testes.workshop_ml_agentes.geocode_address('Rua Sete de Setembro, 66, Rio de Janeiro, Brazil') AS coords
)
SELECT
  coords.latitude,
  coords.longitude
FROM
  geocoded_location;